In [ ]:
! pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"roshansalunke","key":"16ea77830d34a254c4d6bb9ed8b6d87d"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d andrewmvd/animal-faces

100% 693M/696M [00:38<00:00, 17.8MB/s]
100% 696M/696M [00:39<00:00, 18.7MB/s]


In [ ]:
! unzip animal-faces.zip

Streaming output truncated to the last 5000 lines.
  inflating: afhq/train/wild/flickr_wild_001378.jpg  
  inflating: afhq/train/wild/flickr_wild_001379.jpg  
  inflating: afhq/train/wild/flickr_wild_001380.jpg  
  inflating: afhq/train/wild/flickr_wild_001381.jpg  
  inflating: afhq/train/wild/flickr_wild_001382.jpg  
  inflating: afhq/train/wild/flickr_wild_001384.jpg  
  inflating: afhq/train/wild/flickr_wild_001385.jpg  
  inflating: afhq/train/wild/flickr_wild_001386.jpg  
  inflating: afhq/train/wild/flickr_wild_001387.jpg  
  inflating: afhq/train/wild/flickr_wild_001388.jpg  
  inflating: afhq/train/wild/flickr_wild_001389.jpg  
  inflating: afhq/train/wild/flickr_wild_001390.jpg  
  inflating: afhq/train/wild/flickr_wild_001391.jpg  
  inflating: afhq/train/wild/flickr_wild_001392.jpg  
  inflating: afhq/train/wild/flickr_wild_001393.jpg  
  inflating: afhq/train/wild/flickr_wild_001394.jpg  
  inflating: afhq/train/wild/flickr_wild_001395.jpg  
  inflating: afhq/train/wild/fl

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [ ]:
gen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.3,
    channel_shift_range=10,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    vertical_flip = True
)

In [ ]:
train_path = '/content/afhq/train'

In [ ]:
val_path = '/content/afhq/val'

In [ ]:
image_size = 224
batch_size = 32

In [ ]:
train_generator = gen.flow_from_directory(
    train_path,
    target_size=(image_size,image_size),
    subset='training',
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=True,
    class_mode='categorical',
    seed=42
)

Found 14630 images belonging to 3 classes.


In [ ]:
val_generator = gen.flow_from_directory(
    val_path,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=True,
    class_mode='categorical',
    seed=42
)

Found 1500 images belonging to 3 classes.


In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',
    input_shape = (224,224, 3),
    include_top=False
)

base_model.trainable = True

In [ ]:
number_of_classes = 3

In [ ]:
input = tf.keras.Input(shape=(224,224,3))

layer = base_model(input, training=True)

layer = tf.keras.layers.GlobalAveragePooling2D()(layer)

layer = tf.keras.layers.Dropout(0.1)(layer)

output = tf.keras.layers.Dense(number_of_classes,
                               kernel_initializer=tf.keras.initializers.GlorotUniform(seed=43),
                               activation='softmax')(layer)

model = tf.keras.Model(input, output)

In [ ]:
print(model.summary())

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_3  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense_3 (Dense)             (None, 3)                 3843      
                                                                 
Total params: 2261827 (8.63 MB)
Trainable params: 2227715 (

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
step_size_train = train_generator.n//train_generator.batch_size
step_size_valid = val_generator.n//val_generator.batch_size

In [ ]:
model.fit_generator(generator=train_generator,
          steps_per_epoch = step_size_train,
          validation_data=val_generator,
          validation_steps=step_size_valid,
          epochs=5)

<ipython-input-42-b0aa0d025cd5>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/5
457/457 [==============================] - 320s 641ms/step - loss: 0.1348 - accuracy: 0.9552 - val_loss: 0.0881 - val_accuracy: 0.9667
Epoch 2/5
457/457 [==============================] - 289s 631ms/step - loss: 0.0753 - accuracy: 0.9745 - val_loss: 0.0824 - val_accuracy: 0.9708
Epoch 3/5
457/457 [==============================] - 275s 602ms/step - loss: 0.0472 - accuracy: 0.9844 - val_loss: 0.0560 - val_accuracy: 0.9830
Epoch 4/5
457/457 [==============================] - 290s 635ms/step - loss: 0.0452 - accuracy: 0.9851 - val_loss: 0.0661 - val_accuracy: 0.9823
Epoch 5/5
457/457 [==============================] - 289s 632ms/step - loss: 0.0477 - accuracy: 0.9840 - val_loss: 0.0483 - val_accuracy: 0.9857
